# Pretrained BERT model

Use a pretrained BertModel from HuggingFace, only fit the classifier layers

https://github.com/huggingface/transformers/blob/master/notebooks/02-transformers.ipynb

Download distilbert model:
* https://s3.amazonaws.com/models.huggingface.co/bert/distilbert-base-cased-tf_model.h5
* https://s3.amazonaws.com/models.huggingface.co/bert/distilbert-base-cased-config.json

In [2]:
import pandas as pd
from transformers import BertTokenizer

import re

import logging

logging.basicConfig(level=logging.WARNING)

2000 records is 3 minutes for creating the embeddings. If we assume linear performance it would take 75 minutes to convert all embeddings. Unfortantely, it leads to a dead kernel in the tokenize step. We need to create batches to run this on a local machine.

In [3]:
df = pd.read_csv('../data/IMDB Dataset.csv')

SAMPLE_SIZE = 50000

def preprocess_imdb_raw_data(x):
    x = re.sub("<br\\s*/?>", " ", x)
    return x 

X = [preprocess_imdb_raw_data(x) for x in df['review'].values][:SAMPLE_SIZE]

y = df['sentiment'].apply(lambda x: int(x == 'positive')).values[:SAMPLE_SIZE]

df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


# Using a transformers pipeline
Without any additional training

In [39]:
from transformers import pipeline

nlp_sentence_classif = pipeline('sentiment-analysis')

In [4]:
from sklearn.model_selection import train_test_split

# Use the same test set as before
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, )

In [5]:
X_train_idx = [X.index(X_train) for ]

['This film has been compared to the hilarious British comedy "A Fish Called Wanda", although I can\'t see why. The only connection I can find is the Monty Python one (Eric Idle in "Nuns", John Cleese and Michael Palin in "Wanda"). Otherwise the two are incomparable.  Idle and Robbie Coltrane are two gangsters who want to get out of the business before they end up dead, so they decide to rip off their boss and make for Rio. When the getaway goes wrong, the two are forced to take refuge in a convent, as nuns.  What at first promises to be a riot soon becomes a predictable, average movie with the usual tasteless sex jokes and bland humour. Once again there are the occasional high spots, but neither cast nor crew manage to inspire proceedings. An attempt at a "Wanda" type manic finish fails too, along with the effort at "men in drag" humour, which is hardly surprising. When you think about it, men trying to be women is funny, men trying to be nuns is ridiculous.  Friday, April 22, 1994 - 

In [42]:
predicted_sentiment = [nlp_sentence_classif(x)[0]['label'].lower() for x in X_test]

In [43]:
from sklearn.metrics import classification_report

y_pred = [s == 'positive' for s in predicted_sentiment]

print(f"Test: {classification_report(y_test, y_pred)}")

Test:               precision    recall  f1-score   support

           0       0.87      0.92      0.89      5044
           1       0.91      0.86      0.88      4956

    accuracy                           0.89     10000
   macro avg       0.89      0.89      0.89     10000
weighted avg       0.89      0.89      0.89     10000



# Pre-trained BertModel

In [46]:
import torch
from transformers import AutoTokenizer, BertTokenizer
from transformers import TFBertModel, BertModel

torch.set_grad_enabled(False)

Q: Can you use the tokenizer from a different model?

Q: Distilbert also takes around 3 to create embeddings. What is the efficiency gain that we could have expected?

In [48]:
# Store the model we want to use
MODEL_NAME = "bert-base-cased" 

# We need to create the model and tokenizer
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

# MODEL_NAME = "../models/distilbert-base-cased"

model_pt = BertModel.from_pretrained(MODEL_NAME)

model_pt.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(28996, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [ ]:
MAX_SEQ_LENGTH = 200

def get_sentence_embedding(text):
    
    tokenized_text = tokenizer.encode(text, 
                                      return_tensors='pt', 
                                      max_length=MAX_SEQ_LENGTH)

    with torch.no_grad():
        last_hidden_states, _ = model_pt(tokenized_text, )
    
    # Use the mean of the sentence embeddings
    return torch.mean(last_hidden_states, dim=1).numpy().reshape(1, -1)[0]

embeddings = np.concatenate([
  get_sentence_embedding(text)
  for text in X
])

## Generate sentence embeddings per batch
Contains error - no proper results

In [31]:
def generate_embeddings(X, batch_size):
    
    for i in range(0, len(X), batch_size):
    
        tokens = tokenizer.batch_encode_plus(X[i:i+batch_size], 
                                         max_length=MAX_SEQ_LENGTH, 
                                         pad_to_max_length=True,
                                         return_tensors='tf')
        _, pooled = model_tf(tokens)

        yield i, np.array(pooled)

In [ ]:
for (i, embedding) in generate_embeddings(X, 5000):
    np.save(f'../models/bert-model/bert_pooled_layer_{i}.npy', embedding)

### Load embeddings

In [44]:
import os
from sklearn.preprocessing import Normalizer

embeddings = np.concatenate([
    np.load(f'../models/bert-model/bert_pooled_layer_{i}.npy')
    for i in range(0, SAMPLE_SIZE, 5000)
])

In [45]:
normalized_embeddings = Normalizer().fit_transform(embeddings)

In [46]:
X_train, X_test, y_train, y_test = train_test_split(normalized_embeddings, y, test_size=0.2, random_state=1)

## Model for last clf layer

In [47]:
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras import losses
from tensorflow.keras.optimizers import Adam

def make_simple_model(embedding_size=768):

    inp = Input(shape=[embedding_size])
    
    x = Dense(128, activation="relu")(inp)
    
    out = Dense(1, activation="sigmoid")(x)

    model = Model(inp, out)
    
    print(model.summary())
    
    model.compile(Adam(lr=5e-3), loss=losses.binary_crossentropy, metrics=['accuracy'])
    
    return model

model_clf = make_simple_model()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 768)]             0         
_________________________________________________________________
dense_13 (Dense)             (None, 128)               98432     
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 129       
Total params: 98,561
Trainable params: 98,561
Non-trainable params: 0
_________________________________________________________________
None


In [48]:
model_clf.fit(X_train, y_train, epochs=150)

Train on 40000 samples
Epoch 1/150
40000/40000 [==============================] - 2s 39us/sample - loss: 0.6710 - accuracy: 0.5826
Epoch 2/150
40000/40000 [==============================] - 1s 33us/sample - loss: 0.6445 - accuracy: 0.6234
Epoch 3/150
40000/40000 [==============================] - 1s 32us/sample - loss: 0.6269 - accuracy: 0.6482
Epoch 4/150
40000/40000 [==============================] - 1s 34us/sample - loss: 0.6171 - accuracy: 0.6563
Epoch 5/150
40000/40000 [==============================] - 1s 34us/sample - loss: 0.6073 - accuracy: 0.6685
Epoch 6/150
40000/40000 [==============================] - 1s 34us/sample - loss: 0.6076 - accuracy: 0.6680
Epoch 7/150
40000/40000 [==============================] - 1s 35us/sample - loss: 0.6052 - accuracy: 0.6719
Epoch 8/150
40000/40000 [==============================] - 1s 34us/sample - loss: 0.5996 - accuracy: 0.6759
Epoch 9/150
40000/40000 [==============================] - 1s 37us/sample - loss: 0.5976 - accuracy: 0.6800
Epoch

40000/40000 [==============================] - 1s 27us/sample - loss: 0.5376 - accuracy: 0.7247
Epoch 150/150
40000/40000 [==============================] - 1s 27us/sample - loss: 0.5432 - accuracy: 0.7167


## Validation

In [49]:
from sklearn.metrics import classification_report

y_test_probs = model_clf.predict(x=X_test)
y_test_pred = (y_test_probs >= 0.5).astype(int)

print(f"Test: {classification_report(y_test, y_test_pred)}")

Test:               precision    recall  f1-score   support

           0       0.79      0.53      0.63      5044
           1       0.64      0.86      0.73      4956

    accuracy                           0.69     10000
   macro avg       0.72      0.69      0.68     10000
weighted avg       0.72      0.69      0.68     10000



In [50]:
from sklearn.metrics import classification_report

y_train_probs = model_clf.predict(x=X_train)
y_train_pred = (y_train_probs >= 0.5).astype(int)

print(f"Train: {classification_report(y_train, y_train_pred)}")

Train:               precision    recall  f1-score   support

           0       0.80      0.55      0.65     19956
           1       0.66      0.87      0.75     20044

    accuracy                           0.71     40000
   macro avg       0.73      0.71      0.70     40000
weighted avg       0.73      0.71      0.70     40000



# Train with own weights
https://github.com/huggingface/transformers/issues/400#issuecomment-571354368

~~~py
for w in model.bert.weights():
        w._trainable = False
~~~

Train on a GPU, use pytorch https://mccormickml.com/2019/07/22/BERT-fine-tuning/#11-using-colab-gpu-for-training